<a href="https://colab.research.google.com/github/dprzek/esg/blob/main/esg_adash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# biblioteki i pakiety
import numpy as np
import statsmodels.api as sm
from sklearn.cluster import KMeans


Ładowanie syntetycznych danych z załączonej CSV-ki

In [2]:
data = pd.read_csv("esg_data.csv", sep = ';') 
data.head()

,entity_id,year,construction_year,capital_renovation_year,emission_CO2,emission_CH4,emission_SO2,power_thermal,power_electrical,no_boilers_fluidized,no_boilers_water,no_boilers_steam,fuel_coal_mio_t,fuel_gas_mio_m3,fuel_oil_mio_l,filter_coverage
0,1001,2017,1954,2008,232.672663,3.618711,1.097147,1400,386,2,2,1,1.0,500,400,0.89
1,1001,2018,1954,2008,230.728456,3.534175,0.944533,1400,386,2,2,1,1.0,500,400,0.89
2,1001,2019,1954,2008,234.708260,3.674400,1.021560,1400,386,2,2,1,1.0,500,400,0.89
3,1001,2020,1954,2008,257.536767,3.700645,0.987547,1400,386,2,2,1,1.0,500,400,0.89
4,1001,2021,1954,2008,228.861484,4.106222,1.206645,1500,400,2,1,2,1.0,500,440,0.93


**Pierwszy** element analityczny: regresja

In [24]:
model = sm.OLS(data["emission_CO2"],
               sm.add_constant(data[['year', 'construction_year', "capital_renovation_year",
                                    "power_thermal","power_electrical","no_boilers_fluidized",
                                    "no_boilers_water","no_boilers_steam","fuel_coal_mio_t",
                                    "fuel_gas_mio_m3","fuel_oil_mio_l","filter_coverage"]]))
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           emission_CO2   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                     178.4
Date:                Wed, 18 May 2022   Prob (F-statistic):           1.68e-76
Time:                        12:25:52   Log-Likelihood:                -614.60
No. Observations:                 148   AIC:                             1255.
Df Residuals:                     135   BIC:                             1294.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -2

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


**Drugi** element analityczny: wizualizacja kmeans